In [21]:
import pandas as pd
import os

from itables import show

## Part 1

In [68]:
def add_headers(rawdi, outdir):
    for _,_,files in os.walk(rawdir):
        break
    
    for filename in files:
        if filename!='climdiv_county-readme.txt':
            print(os.path.join(rawdir, filename))
            with open(os.path.join(rawdir, filename), 'r') as f, open(os.path.join(outdir, filename), 'w') as o:
                content = f.read()
                line='Code  Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  Nov  Dec'
                o.write(line + '\n' + content)
                
rawdir = 'RawClimDiv'
climdir = 'ClimDivData'
add_headers(rawdir, climdir)

RawClimDiv/climdiv_pcpn_cty.txt
RawClimDiv/climdiv_tmin_st.txt
RawClimDiv/climdiv_tmin_cty.txt
RawClimDiv/climdiv_tavg_st.txt
RawClimDiv/climdiv_tmax_st.txt
RawClimDiv/climdiv_tmax_cty.txt
RawClimDiv/climdiv_pcpn_st.txt
RawClimDiv/climdiv_tavg_cty.txt


In [70]:
#county stuff
for _,_,files in os.walk(climdir):
    break

dfs_dict = {}
for filename in files:
    if not filename.endswith('cty.txt'):
        continue
            
    fpath = os.path.join(climdir, filename)
    dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})

    dict['StateFips'] = dict['Code'].str[0:2]
    dict['CountyFips'] = dict['Code'].str[2:5]
    dict['DataType'] = dict['Code'].str[5:7]
    dict['Year'] = dict['Code'].str[7:12]

    cols = list(dict.columns)
    cols.remove('Year')
    cols.insert(0, 'Year')
    cols.remove('CountyFips')
    cols.insert(0, 'CountyFips')
    cols.remove('StateFips')
    cols.insert(0, 'StateFips')
    cols.remove('DataType')
    cols.insert(0, 'DataType')
    cols.remove('Code')
    dict = dict[cols]

    outpath = os.path.join(climdir, filename.split('.')[0] + '_clean.csv')
    dict.to_csv(outpath, index=False)

/tmp/ipykernel_32880/1504482282.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})
/tmp/ipykernel_32880/1504482282.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})
/tmp/ipykernel_32880/1504482282.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype=

In [76]:
#state stuff
for _,_,files in os.walk(climdir):
    break

dfs_dict = {}
for filename in files:
    if not filename.endswith('st.txt'):
        continue
            
    fpath = os.path.join(climdir, filename)
    dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})

    dict['StateFipsOrRegion'] = dict['Code'].str[0:3]
    dict['Division'] = dict['Code'].str[3:4]
    dict['DataType'] = dict['Code'].str[4:6]
    dict['Year'] = dict['Code'].str[6:10]

    cols = list(dict.columns)
    cols.remove('Year')
    cols.insert(0, 'Year')
    cols.remove('Division')
    cols.insert(0, 'Division')
    cols.remove('StateFipsOrRegion')
    cols.insert(0, 'StateFipsOrRegion')
    cols.remove('DataType')
    cols.insert(0, 'DataType')
    cols.remove('Code')
    dict = dict[cols]

    outpath = os.path.join(climdir, filename.split('.')[0] + '_clean.csv')
    dict.to_csv(outpath, index=False)

/tmp/ipykernel_32880/2159036603.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})
/tmp/ipykernel_32880/2159036603.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype={'Code': 'str'})
/tmp/ipykernel_32880/2159036603.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dict = pd.read_csv(fpath, sep=' +', dtype=